# Quality control of WEAVE RSS files

Wavelength calibration test

Based on sky emission lines identified in Row-stacked spectra (RSS)

# 0. Initialisation

## Imports

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import AutoMinorLocator

import numpy as np
import os
import glob
from time import time
from scipy import ndimage

from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import constants as c
from astropy.table import Table


Plotting functions:

In [ ]:
def new_figure(fig_name, figsize=(10, 5), nrows=1, ncols=1, sharex='col', sharey='row', gridspec_kw={'hspace': 0, 'wspace': 0}):
    plt.close(fig_name)
    fig = plt.figure(fig_name, figsize=figsize)
    axes = fig.subplots(nrows=nrows, ncols=ncols, squeeze=False,
                        sharex=sharex, sharey=sharey,
                        gridspec_kw=gridspec_kw
                       )
    fig.set_tight_layout(True)
    for ax in axes.flat:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')

    fig.suptitle(f'{rss.filename} {fig_name}')
    
    return fig, axes


In [ ]:
default_cmap = plt.get_cmap("gist_earth").copy()
default_cmap.set_bad('gray')


def colour_map(ax, cblabel, data, cmap=default_cmap, norm=None, xlabel=None, x=None, ylabel=None, y=None):
    
    if norm is None:
        percentiles = np.array([1, 16, 50, 84, 99])
        ticks = np.nanpercentile(data, percentiles)
        linthresh = np.median(data[data > 0])
        norm = colors.SymLogNorm(vmin=ticks[0], vmax=ticks[-1], linthresh=linthresh)
    else:
        ticks = None
    if y is None:
        y = np.arange(data.shape[0])
    if x is None:
        x = np.arange(data.shape[1])

    im = ax.imshow(data,
                   extent=(x[0]-(x[1]-x[0])/2, x[-1]+(x[-1]-x[-2])/2, y[0]-(y[1]-y[0])/2, y[-1]+(y[-1]-y[-2])/2),
                   interpolation='nearest', origin='lower',
                   cmap=cmap,
                   norm=norm,
                  )
    ax.set_aspect('auto')
    if xlabel is not None:
        ax.set_xlabel(xlabel)
    if ylabel is not None:
        ax.set_ylabel(ylabel)

    cb = fig.colorbar(im, ax=ax, orientation='vertical', shrink=.9)
    cb.ax.set_ylabel(cblabel)
    if ticks is not None:
        cb.ax.set_yticks(ticks=ticks, labels=[f'{value:.3g} ({percent}%)' for value, percent in zip(ticks, percentiles)])
    cb.ax.tick_params(labelsize='small')
    
    return im, cb


## Ancillary data

UVES sky emission atlas: <https://www.eso.org/observing/dfo/quality/UVES/pipeline/sky_spectrum.html>

In [ ]:
wave_flux = np.empty((0, 2))
filenames = glob.glob('sky/UVES_sky_emission_atlas/gident_*.dat')
filenames.sort()
for filename in filenames:
    print(filename)
    wave_flux = np.concatenate((wave_flux, np.loadtxt(filename, usecols=(1, 4), skiprows=3, comments=['#', '--------'])), axis=0)
UVES_atlas = Table(wave_flux, names=('wavelength', 'flux'))

## RSS files

In [ ]:
class WEAVE_RSS(object):
    
    def __init__(self, filename):
        '''Read a WEAVE "single exposure" file (i.e. row-stacked spectra for just one arm)'''
        self.filename = filename
        self.hdu = fits.open(filename)
        self.wcs = WCS(self.hdu[1].header)
        pixels = np.arange(self.hdu[1].data.shape[1])
        self.wavelength = self.wcs.spectral.array_index_to_world(pixels).to_value(u.Angstrom)
        self.counts = self.hdu[3].data
        self.counts_error = np.where(self.hdu[4].data > 0, 1/np.sqrt(self.hdu[4].data), np.nan)
        self.sky_counts = self.hdu[3].data - self.hdu[1].data
        self.sensitivity_function = self.hdu[5].data
        self.flux = self.hdu[1].data*self.sensitivity_function
        self.sky_sub_ivar = self.hdu[2].data
        
        bad = np.isnan(self.counts_error).nonzero()
        self.counts[bad] = np.nan
        self.sky_counts[bad] = np.nan
        self.sensitivity_function[bad] = np.nan
        self.flux[bad] = np.nan
        self.sky_sub_ivar[bad] = np.nan
        
        self.fibtable = self.hdu[6].data
        #self.sky_fibres = np.where(self.fibtable['TARGUSE'] != 'Patata')
        self.sky_fibres = np.where(self.fibtable['TARGUSE'] == 'S')
        self.target_fibres = np.where(self.fibtable['TARGUSE'] == 'T')
        self.n_sky_fibres = self.sky_fibres[0].size
        self.n_fibres = self.counts.shape[0]

        heliocentric_correction = (1 + np.nanmean(self.fibtable['Helio_cor'])/3e5)
        wave = UVES_atlas['wavelength'] * heliocentric_correction
        inside = np.where((wave > self.wavelength[0]) & (wave < self.wavelength[-1]))
        self.sky_lines = UVES_atlas[inside].copy()
        self.sky_lines['wavelength'] *= heliocentric_correction


In [ ]:
#red_arm = WEAVE_RSS('SV/20230512_13/single_3005695.fit')
#blue_arm = WEAVE_RSS('SV/20230512_13/single_3005696.fit')
#rss_list = [blue_arm, red_arm]
#rss = red_arm
rss = WEAVE_RSS('data/v3/casu.ast.cam.ac.uk/weavedata/1500016316/L1/single_3039342.fit')

In [ ]:
rss.hdu.info()

In [ ]:
#rss.hdu[0].header
#rss.sky_lines

In [ ]:
#rss.fibtable.columns

# 1. Sky spectrum

## First guess: mode and median

Mode as a function of wavelength:

In [ ]:
t0 = time()
n_bins = max(int(np.sqrt(rss.counts.size/rss.wavelength.size)), 101)
percentiles_per_wl = np.linspace(0, 100, n_bins)
percentile_values_per_wl = np.nanpercentile(rss.counts, percentiles_per_wl, axis=0)

print(f"{n_bins} percentiles ({time()-t0:.3g} s)")

In [ ]:
rho_per_wl_x = (percentile_values_per_wl[1:] + percentile_values_per_wl[:-1]) / 2
rho_per_wl = 1 / (percentile_values_per_wl[1:] - percentile_values_per_wl[:-1])
bad = np.where(np.count_nonzero(np.isfinite(rho_per_wl), axis=0) == 0)
rho_per_wl[:, bad] = 0
i_mode = np.nanargmax(rho_per_wl, axis=0)
mode_sky = percentile_values_per_wl[i_mode, np.arange(rss.wavelength.size)]
mode_sky[bad] = np.nan


In [ ]:
n_wavelengths = percentile_values_per_wl.shape[1]
mode_sky = np.empty(n_wavelengths) * np.nan
delta_mode_sky = np.empty(n_wavelengths) * np.nan
for i in range(n_wavelengths):
    rho_x = (percentile_values_per_wl[1:, i] + percentile_values_per_wl[:-1, i]) / 2
    rho = 1 / (percentile_values_per_wl[1:, i] - percentile_values_per_wl[:-1, i])
    if np.count_nonzero(np.isfinite(rho)) > 0:
        peaks = np.where((rho[1:-1] > rho[:-2]) & (rho[1:-1] > rho[2:]))
        valleys = np.where((rho[1:-1] < rho[:-2]) & (rho[1:-1] < rho[2:]))
        max_valley = np.nanmax(rho[1:-1][valleys])
        mode_left, mode_right = np.take(rho_x[rho > max_valley], [1, -1])
        mode_sky[i] = (mode_left + mode_right) / 2
        delta_mode_sky[i] = (mode_right - mode_left) / 2


Median (just for comparison), over the "sky fibres" and the whole dataset

In [ ]:
median_sky_fibres = np.nanmedian(rss.sky_counts[rss.sky_fibres], axis=0)

In [ ]:
i_median = np.argmin(np.abs(percentiles_per_wl-50))
median_sky_all = percentile_values_per_wl[i_median]

Plots:

In [ ]:
fig, axes = new_figure('mode_and_median', nrows=2)

ax = axes[0, 0]
ax.set_ylabel('counts')
ax.set_yscale('log')

ax.plot(rss.wavelength, mode_sky, 'k-', alpha=1, label='mode over all fibres')
ax.fill_between(rss.wavelength, mode_sky-delta_mode_sky, mode_sky+delta_mode_sky, color='k', alpha=.1)
ax.plot(rss.wavelength, median_sky_all, 'r-', alpha=.5, label='median over all fibres')
ax.plot(rss.wavelength, median_sky_fibres, 'b-', alpha=.5, label='median over sky fibres')
ax.legend()

ax = axes[1, 0]
ax.set_ylabel('sky fibre throughput')
ax.set_yscale('log')
ax.plot(rss.wavelength, median_sky_fibres/mode_sky, 'b-', label='median over sky / mode over all')
ax.axhline(np.nanmedian(median_sky_fibres/mode_sky), c='k', ls=':')
ax.legend()

plt.show()

In [ ]:
#np.savetxt('sky_spectrum.txt', np.array([rss.wavelength, mode_sky]).T, fmt='%.1f', header='wavelength, sky_spectrum')

## Line Spread Function (LSF)

Parameters:

In [ ]:
LSF_requested_resolution = 0.01  # Angstrom
LSF_wavelength_range = 20  # Angstrom
LSF_dlambda = np.arange(-LSF_wavelength_range, LSF_wavelength_range+.5*LSF_requested_resolution, LSF_requested_resolution)

Function definitions:

In [ ]:
def normalise(x):
    x -= np.median(x)
    norm = x[x.size//2]
    if norm > 0:
        x /= norm
    else:
        x *= np.nan
    return x


def find_LSF(delta_l, spectrum):
    median_skyline = np.zeros((rss.sky_lines['wavelength'].size, delta_l.size))
    for i, line in enumerate(rss.sky_lines['wavelength']):
        sed = np.interp(line+delta_l, rss.wavelength, spectrum)
        sed = normalise(sed)
        median_skyline[i] = sed
    return normalise(np.nanmedian(median_skyline, axis=0))


def find_FWHM(delta_l, profile):
    threshold = np.max(profile)/2
    left = np.max(delta_l[(delta_l < 0) & (profile < threshold)])
    right = np.min(delta_l[(delta_l > 0) & (profile < threshold)])
    return right-left


In [ ]:
def gaussian_profile(x, mu=0, sigma=1, norm=True):
    g = np.exp(-.5 * ((x-mu) / sigma)**2)
    if norm:
        g /= np.sqrt(2*np.pi) * sigma
    return  g


def refine_Gaussian(x, I, mu0, sigma0):

    good = np.where(np.isfinite(I))
    weight = np.exp(-.5 * ((x[good]-mu0) / sigma0)**2) * (I[good] - np.min(I[good]))
    total_weight = np.sum(weight)

    ivar0 = 1 / sigma0**2
    mu = np.sum(weight * x[good]) / total_weight
    ivar = total_weight / np.sum(weight * (x[good] - mu)**2)

    ivar1 = ivar - ivar0
    mu1 = (mu*ivar - mu0*ivar0) / ivar1
    return mu1, 1/np.sqrt(ivar1)


def fit_Gaussian(x, I, mu0=None, sigma0=None):

    if mu0 is None:
        mu0 = np.nanmean(x)
    if sigma0 is None:
        sigma0 = np.nanstd(x)
    
    delta2 = np.inf
    while delta2 > LSF_requested_resolution**2:
        mu1, sigma1 = refine_Gaussian(x, I, mu0, sigma0)
        delta2 = (mu1 - mu0)**2 + (sigma1 - sigma0)**2
        mu0 = mu1
        sigma0 = sigma1
        #print(mu0, sigma0)
    
    return mu0, sigma0

FWHM of mode-based sky:

In [ ]:
mode_sky_LSF = find_LSF(LSF_dlambda, mode_sky)
mode_sky_FWHM = find_FWHM(LSF_dlambda, mode_sky_LSF)
#LSF_sigma = mode_sky_FWHM / np.sqrt(8*np.log(2))

LSF_mu, LSF_sigma = fit_Gaussian(LSF_dlambda, mode_sky_LSF, 0, mode_sky_FWHM / np.sqrt(8*np.log(2)))

print(f'FWHM of mode-based sky = {mode_sky_FWHM:.3f} (sigma = {mode_sky_FWHM / np.sqrt(8*np.log(2)):.3f}) Angstrom')
print(f'Moments (mu, sigma) = ({LSF_mu:.3f}, {LSF_sigma:.3f}) Angstrom')

In [ ]:
fig, axes = new_figure('sky_LSF')

    
ax = axes[0, 0]
ax.set_ylabel(r'Median Line Spread Function (LSF)')
#ax.set_yscale('log')
#ax.set_ylim(5e-4, 2)


#ax.plot(LSF_dlambda, np.nancumsum(mode_sky_LSF), 'k--', alpha=.5)
#ax.plot(LSF_dlambda, np.nancumsum(mode_sky_LSF*LSF_dlambda)*np.nancumsum(mode_sky_LSF), 'b-', alpha=.5)
ax.plot(LSF_dlambda, mode_sky_LSF, 'k-', alpha=1, label=f'FWHM = {mode_sky_FWHM:.3f} (sigma = {mode_sky_FWHM / np.sqrt(8*np.log(2)):.3f}) $\\AA$ ')
ax.plot(LSF_dlambda, gaussian_profile(LSF_dlambda, LSF_mu, LSF_sigma, False), 'k--', alpha=1, label=f'Gaussian ($\\mu, \\sigma$) = ({LSF_mu:.3f}, {LSF_sigma:.3f}) $\\AA$ ')
ax.axvline(LSF_mu, c='k', ls=':')
ax.legend()

ax = axes[-1, 0]
ax.set_xlabel(r'$(\lambda - \lambda_0)$ [$\AA$]')

In [ ]:
rss.sky_lines

# 3. Individual fibres

## Line-continuum separation

Parameters:

In [ ]:
continuum_peak_separation = 2  # in units of the LSF FWHM

Function definitions:

In [ ]:
'''
def find_peaks(x, y, min_separation):
    peaks = []
    y[np.isnan(y)] = -np.inf
    for i in range(min_separation, y.size-min_separation-1):
        if np.argmax(y[i-min_separation:i+min_separation+1]) == min_separation:
            valleys.append(i)
    y[~np.isfinite(y)] = np.nan
    return peaks
'''

def find_continuum(x, y, min_separation):
    valleys = []
    y[np.isnan(y)] = np.inf
    for i in range(min_separation, y.size-min_separation-1):
        if np.argmin(y[i-min_separation:i+min_separation+1]) == min_separation:
            valleys.append(i)
    y[~np.isfinite(y)] = np.nan
    return np.fmin(y, np.interp(x, x[valleys], y[valleys]))


Find continuum for all fibres:

In [ ]:
t0 = time()
spectral_resolution = (rss.wavelength[-1] - rss.wavelength[0]) / rss.wavelength.size
continuum = np.empty_like(rss.counts)
for i, spectrum in enumerate(rss.counts):
    continuum[i] = find_continuum(rss.wavelength, spectrum, int(continuum_peak_separation*mode_sky_FWHM/spectral_resolution))
print(f"{rss.n_fibres} fibres ({time()-t0:.3g} s)")

Identify strong sky emission lines:

In [ ]:
line_fraction = 1-continuum/rss.counts
line_threshold = np.nanmedian(line_fraction)
#line_threshold = .5

line_mask = np.all(line_fraction > line_threshold, axis=0)
print(f'{np.count_nonzero(line_mask)} out of {rss.wavelength.size} wavelengths identified as strong sky lines (line fraction > {line_threshold:.3f})')

In [ ]:
line_mask[0] = False
line_mask[-1] = False
line_left = np.where(~line_mask[:-1] & line_mask[1:])[0]
line_right = np.where(line_mask[:-1] & ~line_mask[1:])[0]
line_right += 1

Use the sky spectrum as an illustrative example:

In [ ]:
sky_continuum = find_continuum(rss.wavelength, mode_sky, int(continuum_peak_separation*mode_sky_FWHM/spectral_resolution))

In [ ]:
fig, axes = new_figure('sky_continuum')

ax = axes[0, 0]
ax.set_yscale('log')
ax.set_ylim(30, 3e4)

ax.plot(rss.wavelength, mode_sky, 'k-', alpha=.2, label='mode-based sky')
ax.plot(rss.wavelength, sky_continuum, 'k--', alpha=.2, label='continuum')
ax.plot(rss.wavelength, mode_sky - sky_continuum, 'k:', alpha=.2, label='line emission')
ax.legend()

for left, right in zip(line_left, line_right):
    ax.axvspan(rss.wavelength[left], rss.wavelength[right], color='b', alpha=.1)

plt.show()

## Trace lines through fibres

Use the continuum-subtracted sky lines as a reference spectrum:

In [ ]:
reference_spectrum = mode_sky - sky_continuum

In [ ]:
line_reference_wavelength = []
line_fit_weight = []  # used only for the fit
line_reference_intensity = []
for left, right in zip(line_left, line_right):
    weight = (reference_spectrum[left:right])**2
    line_reference_wavelength.append(np.nansum(weight * rss.wavelength[left:right]) / np.nansum(weight))
    line_fit_weight.append(np.nansum(reference_spectrum[left:right]))
    line_reference_intensity.append(np.nanmean(reference_spectrum[left:right]))
line_reference_wavelength = np.array(line_reference_wavelength)
line_fit_weight = np.array(line_fit_weight)
line_reference_intensity = np.array(line_reference_intensity)

Trace line wavelengths for every fibre to compare with the reference spectrum:

In [ ]:
line_fibre_wavelength = []
line_fibre_intensity = []
for left, right in zip(line_left, line_right):
    weight = (rss.counts[:, left:right] - continuum[:, left:right])**2
    line_fibre_wavelength.append(np.nansum(weight * rss.wavelength[np.newaxis, left:right], axis=1) / np.nansum(weight, axis=1))
    line_fibre_intensity.append(np.nanmean(rss.counts[:, left:right] - continuum[:, left:right], axis=1))
line_fibre_wavelength = np.array(line_fibre_wavelength)
line_fibre_intensity = np.array(line_fibre_intensity)

Fit wavelength dependence:

In [ ]:
fit_npoints = 18

In [ ]:
fit_bins = np.linspace(line_reference_wavelength[0], line_reference_wavelength[-1], fit_npoints+1)
line_bin = np.searchsorted(fit_bins, line_reference_wavelength).clip(1, fit_npoints)
line_bin -=1
fit_bin_weight = np.zeros(fit_npoints)
np.add.at(fit_bin_weight, line_bin, line_fit_weight)
line_fit_weight /= fit_bin_weight[line_bin]

In [ ]:
#fit = np.zeros(rss.n_fibres)
fit_points_x = np.zeros(fit_npoints)
np.add.at(fit_points_x, line_bin, line_reference_wavelength*line_fit_weight)
fit_points_y = np.zeros((rss.n_fibres, fit_npoints))
for fibre in range(rss.n_fibres):
    np.add.at(fit_points_y[fibre], line_bin, (line_fibre_wavelength[:, fibre]-line_reference_wavelength)*line_fit_weight)


Plot results:

In [ ]:
random_fibre = np.random.randint(0, rss.n_fibres)
random_fibre = 195

fig, axes = new_figure('line_traces', figsize=(12, 8), nrows=3)

for ax in axes.flat:
    for line in line_left:
        ax.axvline(rss.wavelength[line], c='k', ls=':', alpha=.2)
    for line in line_right:
        ax.axvline(rss.wavelength[line], c='k', ls=':', alpha=.2)
    for line in line_reference_wavelength:
        ax.axvline(line, c='k', ls='--', alpha=.2)

ax = axes[0, 0]
ax.set_yscale('log')
ax.set_ylim(1, 3e4)

ax.plot(rss.wavelength, reference_spectrum, 'b-', alpha=.5, label='reference spectrum')
ax.plot(rss.wavelength, rss.counts[random_fibre] - continuum[random_fibre], 'r-', alpha=.5, label=f'fibre {random_fibre}')

ax.legend()


ax = axes[1, 0]
ax.set_ylabel(r'$\Delta \lambda$ [$\AA$]')
#ax.set_yscale('log')
ax.set_ylim(-.5, .5)

#for fibre in range(rss.n_fibres):
#    ax.plot(line_reference_wavelength, line_fibre_wavelength[:, fibre] - line_reference_wavelength, 'c-', alpha=.02)

ax.plot(line_reference_wavelength, line_fibre_wavelength[:, random_fibre] - line_reference_wavelength, 'r+-', alpha=.5, label=f'fibre {random_fibre}')
ax.legend(loc='lower right')
'''
ax.plot(rss.wavelength, wavelength_correction_zero_point[random_fibre] + wavelength_correction_slope[random_fibre]*rss.wavelength, 'c--',
       label=f'Linear fit to fibre {random_fibre}')
'''
ax.plot(fit_points_x, fit_points_y[random_fibre], 'ks--', alpha=1)

ax.plot(fit_points_x, np.nanmedian(fit_points_y, axis=0), 'b-', alpha=.5)
ax.fill_between(fit_points_x, np.percentile(fit_points_y, 16, axis=0), np.percentile(fit_points_y, 84, axis=0), color='b', alpha=.2)

ax = axes[2, 0]
ax.set_ylim(random_fibre-10, random_fibre+10)

im, cb = colour_map(ax, 'line emission', rss.counts-continuum, xlabel='', x=rss.wavelength)
dummy_cb = fig.colorbar(None, ax=axes[0, 0], orientation='vertical', shrink=.9)
dummy_cb.ax.clear()
dummy_cb.ax.axis('off')
dummy_cb = fig.colorbar(im, ax=axes[1, 0], orientation='vertical', shrink=.9)
dummy_cb.ax.clear()
dummy_cb.ax.axis('off')

ax.axhline(random_fibre, c='k', ls=':', alpha=.4)
for line in line_fibre_wavelength:
    ax.plot(line, np.arange(rss.n_fibres), 'k-', alpha=.4)


ax = axes[-1, 0]
ax.set_xlabel(r'wavelength [$\AA$]')
plt.show()

## Wavelength correction

In [ ]:
line_offset = line_fibre_wavelength - line_reference_wavelength[:, np.newaxis]
line_offset -= np.nanmedian(line_offset, axis=1)[:, np.newaxis]
p16_line_offset, median_line_offset, p84_line_offset = np.nanpercentile(line_offset, [16, 50, 84], axis=0)
model_line_offset = line_offset - median_line_offset[np.newaxis, :]
model_line_offset = median_line_offset[np.newaxis, :] + ndimage.gaussian_filter(model_line_offset, 2)

In [ ]:
p16_corrected_offset, median_corrected_offset, p84_corrected_offset = np.nanpercentile(line_offset - model_line_offset, [16, 50, 84], axis=0)

In [ ]:
fig, axes = new_figure('wavelength_correction', figsize=(10, 8), nrows=2)

ax = axes[0, 0]
ax.set_ylabel('$\Delta\lambda$ [$\AA$]')
ax.plot(median_line_offset, 'k-', alpha=.5, label='original')
ax.fill_between(np.arange(rss.n_fibres), p16_line_offset, p84_line_offset, color='k', alpha=.1)
#ax.plot(median_corrected_offset, 'b-', alpha=.5, label='corrected')
#ax.fill_between(np.arange(rss.n_fibres), p16_corrected_offset, p84_corrected_offset, color='b', alpha=.1)
#ax.legend()
for line in rss.sky_fibres[0]:
    ax.axvline(line, c='b', ls='-', alpha=.2)
cb = fig.colorbar(None, ax=ax)
cb.remove()


ax = axes[1, 0]
im, cb = colour_map(ax, '$\Delta\lambda$ [$\AA$]', line_offset,
                    xlabel='fibre ID (position in detector?)', ylabel='line ID (increasing $\lambda$)', cmap='turbo', norm=colors.Normalize(vmin=-.2, vmax=.2))
ax.scatter(rss.sky_fibres, -np.ones_like(rss.sky_fibres), s=100, marker='+', c='k', label='sky fibres')
ax.scatter(rss.sky_fibres, line_reference_intensity.size*np.ones_like(rss.sky_fibres), s=100, marker='+', c='k')
ax.legend()
ax.set_ylim(-2, line_reference_intensity.size+2)

'''
ax = axes[2, 0]
im, cb = colour_map(ax, '$\Delta\lambda$ - median $\Delta\lambda$ [$\AA$]', model_line_offset,
                    xlabel='fibre ID (position in detector?)', ylabel='line ID (increasing $\lambda$)', cmap='turbo', norm=colors.Normalize(vmin=-.2, vmax=.2))
ax.scatter(rss.sky_fibres, -np.ones_like(rss.sky_fibres), s=100, marker='+', c='k', label='sky fibres')
ax.scatter(rss.sky_fibres, line_reference_intensity.size*np.ones_like(rss.sky_fibres), s=100, marker='+', c='k')
ax.legend()
ax.set_ylim(-2, line_reference_intensity.size+2)


ax = axes[3, 0]
im, cb = colour_map(ax, '$\Delta\lambda$ - median $\Delta\lambda$ [$\AA$]', line_offset - model_line_offset,
                    xlabel='fibre ID (position in detector?)', ylabel='line ID (increasing $\lambda$)', cmap='turbo', norm=colors.Normalize(vmin=-.2, vmax=.2))
ax.scatter(rss.sky_fibres, -np.ones_like(rss.sky_fibres), s=100, marker='+', c='k', label='sky fibres')
ax.scatter(rss.sky_fibres, line_reference_intensity.size*np.ones_like(rss.sky_fibres), s=100, marker='+', c='k')
ax.legend()
ax.set_ylim(-2, line_reference_intensity.size+2)
'''


plt.show()
plt.savefig(f'{rss.filename}-wave_corr.pdf')

In [ ]:
fig, axes = new_figure('relative_offset')
ax = axes[0, 0]
ax.set_ylabel('number of fibres')
ax.hist(median_line_offset, np.linspace(-.25, .25, 61))
ax.set_xlabel(r'median offset $\Delta \lambda\ [\AA]$')
#ax.hist((median_line_offset/6700)*3e5, np.linspace(-15, 15, 61))
#ax.set_xlabel(r'median velocity correction at 6700 $\AA$ [km/s]')
plt.show()
plt.savefig(f'{rss.filename}-relative_ofset.pdf')
np.savetxt(f'{rss.filename}-relative_offset.txt', median_fibre_flat, fmt='%.4f', header='relative_throughput')

## Fibre flat

In [ ]:
line_reference_wavelength.shape, line_fibre_intensity.shape

In [ ]:
fibre_flat = line_fibre_intensity / line_reference_intensity[:, np.newaxis]
fibre_flat /= np.nanmedian(fibre_flat, axis=1)[:, np.newaxis]
p16_fibre_flat, median_fibre_flat, p84_fibre_flat = np.nanpercentile(fibre_flat, [16, 50, 84], axis=0)

In [ ]:
fig, axes = new_figure('fibre_flat', figsize=(10, 8), nrows=2)

ax = axes[0, 0]
ax.set_ylabel('fibre flat')
ax.plot(median_fibre_flat, 'k-', alpha=.5)
ax.fill_between(np.arange(rss.n_fibres), p16_fibre_flat, p84_fibre_flat, color='k', alpha=.1)
for line in rss.sky_fibres[0]:
    ax.axvline(line, c='b', ls='-', alpha=.2)
cb = fig.colorbar(None, ax=ax)
cb.remove()


ax = axes[1, 0]
im, cb = colour_map(ax, 'fibre flat', fibre_flat,
                    xlabel='fibre ID (position in detector?)', ylabel='line ID (increasing $\lambda$)', cmap='turbo', norm=colors.Normalize(vmin=.8, vmax=1.2))
ax.scatter(rss.sky_fibres, -np.ones_like(rss.sky_fibres), s=100, marker='+', c='k', label='sky fibres')
ax.scatter(rss.sky_fibres, line_reference_intensity.size*np.ones_like(rss.sky_fibres), s=100, marker='+', c='k')
ax.legend()
ax.set_ylim(-2, line_reference_intensity.size+2)


plt.show()
plt.savefig(f'{rss.filename}-fibre_flat.pdf')

In [ ]:
fig, axes = new_figure('relative_throughput')
ax = axes[0, 0]
ax.set_ylabel('number of fibres')
ax.hist(median_fibre_flat, np.linspace(.8, 1.2, 81))
ax.set_xlabel(r'relative fibre throughput (median fibre flat)')
plt.show()
plt.savefig(f'{rss.filename}-relative_throughput.pdf')
np.savetxt(f'{rss.filename}-relative_throughput.txt', median_fibre_flat, fmt='%.4f', header='relative_throughput')